#Investigating the results of our Bronze Layer Ingestion and Subsequent Data Cleaning and Aggregation




In [0]:
%sql
SELECT * FROM delta.`dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue` LIMIT 5

Row_ID,Store_ID,Year,Quarter,Total_Revenue,_rescued_data,COGS,Profit
0,3,2023,Q1,46291.27495198135,null,null,null
1,3,2023,Q1,18737.659501830232,null,null,null
2,3,2023,Q1,22846.078536385416,null,null,null
12,5,2023,Q1,21314.800491345446,null,null,null
13,5,2023,Q1,17467.744345224746,null,null,null


In [0]:
%sql
SELECT quarter, count(*) FROM delta.`dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue` GROUP BY quarter;

--SELECT * FROM delta.`dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue`

--DELETE FROM delta.`dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue`


quarter,count(1)
Q2,1779
Q1,1737
Q3,1533
Q4,1529
null,4547


We can also use the describe history command to show us each update to the table.

In [0]:
%sql
DESCRIBE HISTORY delta.`dbfs:/FileStore/tables/TraderJoesRevenue/bronze_tj_fact_revenue`

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2024-08-27T13:35:21Z,7971536220129282,matthew.resner@group1001.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> eca152c9-97c9-4cfc-a78f-a7c10a67f272, epochId -> 1, statsOnLoad -> false)",null,List(3309121777485539),0801-194747-f0qk6m1o,3,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 6111, numOutputBytes -> 44980, numAddedFiles -> 1)",null,Databricks-Runtime/15.3.x-cpu-ml-scala2.12
3,2024-08-27T13:35:18Z,7971536220129282,matthew.resner@group1001.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> eca152c9-97c9-4cfc-a78f-a7c10a67f272, epochId -> -1, statsOnLoad -> false)",null,List(3309121777485539),0801-194747-f0qk6m1o,2,WriteSerializable,true,Map(),null,Databricks-Runtime/15.3.x-cpu-ml-scala2.12
2,2024-08-27T13:33:14Z,7971536220129282,matthew.resner@group1001.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> eca152c9-97c9-4cfc-a78f-a7c10a67f272, epochId -> 0, statsOnLoad -> false)",null,List(3309121777485539),0801-194747-f0qk6m1o,1,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 1549, numOutputBytes -> 37432, numAddedFiles -> 1)",null,Databricks-Runtime/15.3.x-cpu-ml-scala2.12
1,2024-08-27T13:33:04Z,7971536220129282,matthew.resner@group1001.com,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> eca152c9-97c9-4cfc-a78f-a7c10a67f272, epochId -> -1, statsOnLoad -> false)",null,List(3309121777485539),0801-194747-f0qk6m1o,0,WriteSerializable,true,Map(),null,Databricks-Runtime/15.3.x-cpu-ml-scala2.12
0,2024-08-27T13:31:38Z,7971536220129282,matthew.resner@group1001.com,CREATE TABLE,"Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {""delta.enableDeletionVectors"":""true""}, statsOnLoad -> false)",null,List(285342671664234),0801-194747-f0qk6m1o,null,WriteSerializable,true,Map(),null,Databricks-Runtime/15.3.x-cpu-ml-scala2.12


What if we want to report on or record schema evolution to the business? Well then we just need to check the delta log files!

In [0]:
%sql

SELECT * FROM tj_db.silver_tj_fact_revenue
LIMIT 10

STORE_ID,Year,Quarter,Total_Revenue,COGS,Profit
733,2023,Q1,20658.73557159825,20025.0,633.7355716
746,2023,Q1,16698.026233745455,null,null
779,2023,Q1,17862.183504272398,null,null
752,2023,Q1,19732.001376954882,null,null
752,2023,Q1,30435.33610053258,26790.0,3645.336101
762,2023,Q1,49113.23130254069,36626.0,12487.2313
774,2023,Q1,27814.73015050899,27698.0,116.7301505
742,2023,Q1,17170.135138271748,null,null
772,2023,Q1,40074.793396835026,30227.0,9847.793397
741,2023,Q1,16235.499707974266,null,null


In [0]:
%sql
SELECT * FROM tj_db.silver_tj_fact_revenue_profit_recalc
LIMIT 10

STORE_ID,Year,Quarter,Total_Revenue,COGS,Profit
733,2023,Q1,20658.74,20025.0,633.74
746,2023,Q1,16698.03,null,null
779,2023,Q1,17862.18,null,null
752,2023,Q1,19732.0,null,null
752,2023,Q1,30435.34,26790.0,3645.34
762,2023,Q1,49113.23,36626.0,12487.23
774,2023,Q1,27814.73,27698.0,116.73
742,2023,Q1,17170.14,null,null
772,2023,Q1,40074.79,30227.0,9847.79
741,2023,Q1,16235.5,null,null


In [0]:
%sql
SELECT * FROM tj_db.gold_fact_revenue_by_quarter
LIMIT 10

Year,Quarter,Total_Revenue,COGS,Profit
2023,Q1,4.997046588E7,3929192.0,993693.0
